In [ ]:
import socketserver
import threading
import os # os.path.getsize(path)
 
HOST = ''
PORT = 9100
lock = threading.Lock() # syncronized 동기화 진행하는 스레드 생성
 
class UserManager: # 사용자관리 및 채팅 메세지 전송을 담당하는 클래스
                   # ① 채팅 서버로 입장한 사용자의 등록
                   # ② 채팅을 종료하는 사용자의 퇴장 관리
                   # ③ 사용자가 입장하고 퇴장하는 관리
                   # ④ 사용자가 입력한 메세지를 채팅 서버에 접속한 모두에게 전송
 
    def __init__(self):
        self.users = {} # 사용자의 등록 정보를 담을 사전 {사용자 이름:(소켓,주소),...}
 
    def addUser(self, username, conn, addr): # 사용자 ID를 self.users에 추가하는 함수
        if username in self.users: # 이미 등록된 사용자라면
            conn.send('이미 등록된 사용자입니다.\n'.encode())
            return None
 
              # 새로운 사용자를 등록함
        lock.acquire() # 스레드 동기화를 막기위한 락
        self.users[username] = (conn, addr)
        lock.release() # 업데이트 후 락 해제

        self.sendMessageToAll('[%s]님이 입장했습니다. \n' %username)
        print('+++ 대화 참여자 수 [%d]' %len(self.users))

        return username
 
    def removeUser(self, username): #사용자를 제거하는 함수
        if username not in self.users:
            return

        lock.acquire()
        del self.users[username]
        lock.release()
 
        self.sendMessageToAll('[%s]님이 퇴장했습니다.' %username)
        print('--- 대화 참여자 수 [%d]' %len(self.users))
 
    def messageHandler(self, username, msg): # 전송한 msg를 처리하는 부분
        
        if msg.split('@')[0] == 'image_send_server_to_client':
            print('파일 전송 요청을 받았습니다 server -> client')
            file_name = msg.split('@')[1].strip()
            file_size = str(os.path.getsize(file_name))
            print('파일 이름 : ' + file_name)
            print('파일 길이 : ' + file_size)
            #if not exists(file_name): # 파일이 해당 디렉터리에 존재하지 않으면
            #    return # handle()함수를 빠져 나온다.
            self.sendMessageToAll('image_send_server_to_client@stoc_image.jpg@' + file_size + "\n")
            data_transferred = 0
            with open(file_name, 'rb') as f:
                try:
                    data = f.read(1024) # 파일을 1024바이트 읽음
                    while data: # 파일이 빈 문자열일때까지 반복
                        data_transferred += self.sendFileToAll(data)
                        #self.sendMessageToAll(msg)
                        data = f.read(1024)
                except Exception as e:
                    print(e)

            print('전송완료[%s], 전송량[%d]' %(file_name,data_transferred))
    
        else: #msg[0] != '/': # 보낸 메세지의 첫문자가 '/'가 아니면
            self.sendMessageToAll('[%s] %s' %(username, msg))
            return
 
        #if msg.strip() == '/quit': # 보낸 메세지가 'quit'이면
        #    self.removeUser(username)
        #    return -1
 
    def sendMessageToAll(self, msg):
        for conn, addr in self.users.values():
            print('뿌린 메시지 : ' + msg ,end='')
            conn.send(msg.encode())
    
    def sendFileToAll(self, msg):
        total = 0
        for conn, addr in self.users.values():
            print('전송중 : ' + str(msg))
            total += conn.send(msg)
        return total
    
           
 
class MyTcpHandler(socketserver.BaseRequestHandler):
    userman = UserManager()
     
    def handle(self): # 클라이언트가 접속시 클라이언트 주소 출력
        print('[%s] 연결됨' %self.client_address[0])
 
        try:
            username = self.registerUsername()
            msg = self.request.recv(1024)
            while msg:
                msg = msg.decode()
                if msg.split('@')[0] == 'image_send_client_to_server':
                    print('파일 전송 요청을 받았습니다 client -> server')
                    file_name = msg.split('@')[1].strip()
                    file_length_str = msg.split('@')[2].strip()
                    file_length_int = int(file_length_str)
                    print('파일 이름 : ' + file_name)
                    print('파일 길이 : ' + file_length_str)
                    file_size_count = 0
                   
                    with open(file_name, 'wb') as f:
                        try:
                            data = self.request.recv(1024)
                            while data:
                                #print('수신중 : ' + str(data))
                                file_size_count += f.write(data)
                                print('받은 데이터 길이 : ' + str(file_size_count))
                                if file_size_count >= file_length_int:
                                    break
                                data = self.request.recv(1024)
                            print('파일 수신 완료 : ' + str(file_size_count))
                        except Exception as e:
                            print(e)

                else: 
                    if self.userman.messageHandler(username, msg) == -1:
                        self.request.close()
                        break
                msg = self.request.recv(1024)
                 
        except Exception as e:
            print(e)
 
        print('[%s] 접속종료' %self.client_address[0])
        self.userman.removeUser(username)
 
    def registerUsername(self):
        while True:
            username = self.request.recv(1024)
            username = username.decode().strip()
            if self.userman.addUser(username, self.request, self.client_address):
                return username
 

class ChatingServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
        pass
         
def runServer():
    print('+++ 채팅 서버를 시작합니다.')
    print('+++ 채팅 서버를 끝내려면 Ctrl-C를 누르세요.')
 
    try:
        server = ChatingServer((HOST, PORT), MyTcpHandler)
        server.serve_forever()
    except KeyboardInterrupt:
        print('--- 채팅 서버를 종료합니다.')
        server.shutdown()
        server.server_close()

        
runServer()



+++ 채팅 서버를 시작합니다.
+++ 채팅 서버를 끝내려면 Ctrl-C를 누르세요.
[192.168.0.6] 연결됨
뿌린 메시지 : [ㄴㅁ]님이 입장했습니다. 
+++ 대화 참여자 수 [1]
뿌린 메시지 : [ㄴㅁ] 
파일 전송 요청을 받았습니다 client -> server
파일 이름 : ctos_image.jpg
파일 길이 : 700318
받은 데이터 길이 : 1024
받은 데이터 길이 : 2048
받은 데이터 길이 : 2920
받은 데이터 길이 : 3944
받은 데이터 길이 : 4968
받은 데이터 길이 : 5992
받은 데이터 길이 : 7016
받은 데이터 길이 : 8040
받은 데이터 길이 : 8760
받은 데이터 길이 : 9784
받은 데이터 길이 : 10808
받은 데이터 길이 : 11832
받은 데이터 길이 : 12856
받은 데이터 길이 : 13880
받은 데이터 길이 : 14904
받은 데이터 길이 : 15928
받은 데이터 길이 : 16060
받은 데이터 길이 : 17084
받은 데이터 길이 : 18108
받은 데이터 길이 : 19132
받은 데이터 길이 : 20156
받은 데이터 길이 : 21180
받은 데이터 길이 : 22204
받은 데이터 길이 : 23228
받은 데이터 길이 : 24252
받은 데이터 길이 : 24820
받은 데이터 길이 : 25844
받은 데이터 길이 : 26868
받은 데이터 길이 : 27892
받은 데이터 길이 : 28916
받은 데이터 길이 : 29940
받은 데이터 길이 : 30964
받은 데이터 길이 : 31988
받은 데이터 길이 : 33012
받은 데이터 길이 : 34036
받은 데이터 길이 : 35060
받은 데이터 길이 : 36084
받은 데이터 길이 : 36500
받은 데이터 길이 : 37524
받은 데이터 길이 : 38548
받은 데이터 길이 : 39572
받은 데이터 길이 : 40596
받은 데이터 길이 : 41620
받은 데이터 길이 : 42644
받은 데이터 길이 : 43668
받은 데이터

받은 데이터 길이 : 466676
받은 데이터 길이 : 467700
받은 데이터 길이 : 468724
받은 데이터 길이 : 469748
받은 데이터 길이 : 470772
받은 데이터 길이 : 471796
받은 데이터 길이 : 472820
받은 데이터 길이 : 473844
받은 데이터 길이 : 474868
받은 데이터 길이 : 475892
받은 데이터 길이 : 476916
받은 데이터 길이 : 477940
받은 데이터 길이 : 478964
받은 데이터 길이 : 479988
받은 데이터 길이 : 481012
받은 데이터 길이 : 482036
받은 데이터 길이 : 483060
받은 데이터 길이 : 484084
받은 데이터 길이 : 485108
받은 데이터 길이 : 486132
받은 데이터 길이 : 487156
받은 데이터 길이 : 488180
받은 데이터 길이 : 489204
받은 데이터 길이 : 490228
받은 데이터 길이 : 491252
받은 데이터 길이 : 492276
받은 데이터 길이 : 493300
받은 데이터 길이 : 494324
받은 데이터 길이 : 495348
받은 데이터 길이 : 496372
받은 데이터 길이 : 497396
받은 데이터 길이 : 498420
받은 데이터 길이 : 499444
받은 데이터 길이 : 500468
받은 데이터 길이 : 501492
받은 데이터 길이 : 502516
받은 데이터 길이 : 503540
받은 데이터 길이 : 504564
받은 데이터 길이 : 505588
받은 데이터 길이 : 506612
받은 데이터 길이 : 507636
받은 데이터 길이 : 508660
받은 데이터 길이 : 509684
받은 데이터 길이 : 510708
받은 데이터 길이 : 511732
받은 데이터 길이 : 512756
받은 데이터 길이 : 513780
받은 데이터 길이 : 514804
받은 데이터 길이 : 515828
받은 데이터 길이 : 516852
받은 데이터 길이 : 517876
받은 데이터 길이 : 518900
받은 데이터 길이 : 

파일 전송 요청을 받았습니다 client -> server
파일 이름 : ctos_image.jpg
파일 길이 : 700318
받은 데이터 길이 : 1024
받은 데이터 길이 : 2048
받은 데이터 길이 : 3072
받은 데이터 길이 : 4096
받은 데이터 길이 : 5120
받은 데이터 길이 : 6144
받은 데이터 길이 : 7168
받은 데이터 길이 : 8192
받은 데이터 길이 : 9216
받은 데이터 길이 : 10240
받은 데이터 길이 : 11264
받은 데이터 길이 : 12288
받은 데이터 길이 : 13312
받은 데이터 길이 : 14336
받은 데이터 길이 : 15360
받은 데이터 길이 : 16384
받은 데이터 길이 : 17408
받은 데이터 길이 : 18432
받은 데이터 길이 : 19456
받은 데이터 길이 : 20480
받은 데이터 길이 : 21504
받은 데이터 길이 : 22528
받은 데이터 길이 : 23552
받은 데이터 길이 : 24576
받은 데이터 길이 : 25600
받은 데이터 길이 : 26624
받은 데이터 길이 : 27648
받은 데이터 길이 : 28672
받은 데이터 길이 : 29696
받은 데이터 길이 : 30720
받은 데이터 길이 : 31744
받은 데이터 길이 : 32768
받은 데이터 길이 : 33792
받은 데이터 길이 : 34816
받은 데이터 길이 : 35840
받은 데이터 길이 : 36864
받은 데이터 길이 : 37888
받은 데이터 길이 : 38912
받은 데이터 길이 : 39936
받은 데이터 길이 : 40960
받은 데이터 길이 : 41984
받은 데이터 길이 : 43008
받은 데이터 길이 : 44032
받은 데이터 길이 : 45056
받은 데이터 길이 : 46080
받은 데이터 길이 : 46720
받은 데이터 길이 : 47744
받은 데이터 길이 : 48768
받은 데이터 길이 : 49792
받은 데이터 길이 : 50816
받은 데이터 길이 : 51840
받은 데이터 길이 : 52864
받은

받은 데이터 길이 : 449244
받은 데이터 길이 : 449680
받은 데이터 길이 : 450704
받은 데이터 길이 : 451728
받은 데이터 길이 : 452752
받은 데이터 길이 : 453776
받은 데이터 길이 : 454800
받은 데이터 길이 : 455824
받은 데이터 길이 : 456848
받은 데이터 길이 : 457872
받은 데이터 길이 : 458896
받은 데이터 길이 : 459920
받은 데이터 길이 : 460944
받은 데이터 길이 : 461968
받은 데이터 길이 : 462992
받은 데이터 길이 : 464016
받은 데이터 길이 : 465040
받은 데이터 길이 : 466064
받은 데이터 길이 : 467088
받은 데이터 길이 : 468112
받은 데이터 길이 : 469136
받은 데이터 길이 : 470160
받은 데이터 길이 : 471184
받은 데이터 길이 : 472208
받은 데이터 길이 : 473232
받은 데이터 길이 : 474256
받은 데이터 길이 : 475280
받은 데이터 길이 : 476304
받은 데이터 길이 : 477328
받은 데이터 길이 : 478352
받은 데이터 길이 : 479376
받은 데이터 길이 : 480400
받은 데이터 길이 : 481424
받은 데이터 길이 : 482448
받은 데이터 길이 : 483472
받은 데이터 길이 : 484496
받은 데이터 길이 : 485520
받은 데이터 길이 : 486544
받은 데이터 길이 : 487568
받은 데이터 길이 : 488592
받은 데이터 길이 : 489616
받은 데이터 길이 : 490640
받은 데이터 길이 : 491664
받은 데이터 길이 : 492688
받은 데이터 길이 : 493712
받은 데이터 길이 : 494736
받은 데이터 길이 : 495760
받은 데이터 길이 : 496784
받은 데이터 길이 : 497808
받은 데이터 길이 : 498832
받은 데이터 길이 : 499856
받은 데이터 길이 : 500880
받은 데이터 길이 : 

파일 전송 요청을 받았습니다 server -> client
파일 이름 : stoc_image.jpg
파일 길이 : 4720
뿌린 메시지 : image_send_server_to_client@stoc_image.jpg@4720
전송중 : b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x13\x12\x12\x15\x12\x12\x12\x12\x15\x10\x12\x15\x12\x15\x16\x10\x15\x15\x10\x15\x15\x10\x15\x15\x12\x16\x16\x15\x15\x15\x15\x18\x1d( \x18\x1a%\x1b\x15\x15!1!%)+...\x17\x1f3:3-7(9.+\x01\n\n\n\x0e\r\x0e\x1a\x10\x10\x17+\x1d\x1d\x1d-.---+-+---------+-.---------++-------------------\xff\xc0\x00\x11\x08\x00\xb4\x01\x17\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x03\x00\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x04\x06\x07\x05\x03\x08\xff\xc4\x00I\x10\x00\x02\x01\x02\x03\x05\x05\x05\x03\x07\x08\n\x03\x00\x00\x00\x00\x01\x02\x03\x11\x04\x121\x05!AQa\x07\x13q\x81\x91\x06\xa1\xb1\xc1\xf0\x14"\xd1\x15#2rs\xb4\xf1BR\x82\x84\x93\xb2\xb3\xe1$%&56CTt\xc2\xd2\x1734\xff\xc4\x00\x1a\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\

파일 전송 요청을 받았습니다 server -> client
파일 이름 : stoc_image.jpg
파일 길이 : 4720
뿌린 메시지 : image_send_server_to_client@stoc_image.jpg@4720
전송중 : b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x13\x12\x12\x15\x12\x12\x12\x12\x15\x10\x12\x15\x12\x15\x16\x10\x15\x15\x10\x15\x15\x10\x15\x15\x12\x16\x16\x15\x15\x15\x15\x18\x1d( \x18\x1a%\x1b\x15\x15!1!%)+...\x17\x1f3:3-7(9.+\x01\n\n\n\x0e\r\x0e\x1a\x10\x10\x17+\x1d\x1d\x1d-.---+-+---------+-.---------++-------------------\xff\xc0\x00\x11\x08\x00\xb4\x01\x17\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x03\x00\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x04\x06\x07\x05\x03\x08\xff\xc4\x00I\x10\x00\x02\x01\x02\x03\x05\x05\x05\x03\x07\x08\n\x03\x00\x00\x00\x00\x01\x02\x03\x11\x04\x121\x05!AQa\x07\x13q\x81\x91\x06\xa1\xb1\xc1\xf0\x14"\xd1\x15#2rs\xb4\xf1BR\x82\x84\x93\xb2\xb3\xe1$%&56CTt\xc2\xd2\x1734\xff\xc4\x00\x1a\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\

파일 전송 요청을 받았습니다 client -> server
파일 이름 : ctos_image.jpg
파일 길이 : 700318
받은 데이터 길이 : 1024
받은 데이터 길이 : 2048
받은 데이터 길이 : 3072
받은 데이터 길이 : 4096
받은 데이터 길이 : 5120
받은 데이터 길이 : 6144
받은 데이터 길이 : 7168
받은 데이터 길이 : 8192
받은 데이터 길이 : 9216
받은 데이터 길이 : 10240
받은 데이터 길이 : 11264
받은 데이터 길이 : 12288
받은 데이터 길이 : 13140
받은 데이터 길이 : 14164
받은 데이터 길이 : 14600
받은 데이터 길이 : 15624
받은 데이터 길이 : 16648
받은 데이터 길이 : 17672
받은 데이터 길이 : 18696
받은 데이터 길이 : 19720
받은 데이터 길이 : 20744
받은 데이터 길이 : 21768
받은 데이터 길이 : 22792
받은 데이터 길이 : 23816
받은 데이터 길이 : 24840
받은 데이터 길이 : 25864
받은 데이터 길이 : 26888
받은 데이터 길이 : 27912
받은 데이터 길이 : 28936
받은 데이터 길이 : 29960
받은 데이터 길이 : 30984
받은 데이터 길이 : 32008
받은 데이터 길이 : 32120
받은 데이터 길이 : 33144
받은 데이터 길이 : 33580
받은 데이터 길이 : 34604
받은 데이터 길이 : 35628
받은 데이터 길이 : 36652
받은 데이터 길이 : 37676
받은 데이터 길이 : 38700
받은 데이터 길이 : 39420
받은 데이터 길이 : 40444
받은 데이터 길이 : 41468
받은 데이터 길이 : 42492
받은 데이터 길이 : 43516
받은 데이터 길이 : 44540
받은 데이터 길이 : 45564
받은 데이터 길이 : 46588
받은 데이터 길이 : 47612
받은 데이터 길이 : 48636
받은 데이터 길이 : 49660
받은 데이터 길이 : 50684
받은

받은 데이터 길이 : 531004
받은 데이터 길이 : 532028
받은 데이터 길이 : 533052
받은 데이터 길이 : 534076
받은 데이터 길이 : 535100
받은 데이터 길이 : 536124
받은 데이터 길이 : 537148
받은 데이터 길이 : 538172
받은 데이터 길이 : 539196
받은 데이터 길이 : 540220
받은 데이터 길이 : 541244
받은 데이터 길이 : 542268
받은 데이터 길이 : 543292
받은 데이터 길이 : 544316
받은 데이터 길이 : 545340
받은 데이터 길이 : 546364
받은 데이터 길이 : 547388
받은 데이터 길이 : 548412
받은 데이터 길이 : 549436
받은 데이터 길이 : 550460
받은 데이터 길이 : 551484
받은 데이터 길이 : 552508
받은 데이터 길이 : 553532
받은 데이터 길이 : 554556
받은 데이터 길이 : 555580
받은 데이터 길이 : 556260
받은 데이터 길이 : 557284
받은 데이터 길이 : 558308
받은 데이터 길이 : 559180
받은 데이터 길이 : 560204
받은 데이터 길이 : 561228
받은 데이터 길이 : 562252
받은 데이터 길이 : 563276
받은 데이터 길이 : 564300
받은 데이터 길이 : 565324
받은 데이터 길이 : 566348
받은 데이터 길이 : 567372
받은 데이터 길이 : 568396
받은 데이터 길이 : 569420
받은 데이터 길이 : 570444
받은 데이터 길이 : 571468
받은 데이터 길이 : 572492
받은 데이터 길이 : 573516
받은 데이터 길이 : 574540
받은 데이터 길이 : 575564
받은 데이터 길이 : 576588
받은 데이터 길이 : 577612
받은 데이터 길이 : 578636
받은 데이터 길이 : 579660
받은 데이터 길이 : 580684
받은 데이터 길이 : 581708
받은 데이터 길이 : 582732
받은 데이터 길이 : 

받은 데이터 길이 : 279728
받은 데이터 길이 : 280752
받은 데이터 길이 : 281776
받은 데이터 길이 : 282800
받은 데이터 길이 : 283824
받은 데이터 길이 : 284848
받은 데이터 길이 : 285872
받은 데이터 길이 : 286896
받은 데이터 길이 : 287920
받은 데이터 길이 : 288944
받은 데이터 길이 : 289968
받은 데이터 길이 : 290992
받은 데이터 길이 : 292016
받은 데이터 길이 : 293040
받은 데이터 길이 : 294064
받은 데이터 길이 : 295088
받은 데이터 길이 : 296112
받은 데이터 길이 : 297136
받은 데이터 길이 : 298160
받은 데이터 길이 : 299184
받은 데이터 길이 : 300208
받은 데이터 길이 : 301232
받은 데이터 길이 : 302256
받은 데이터 길이 : 303280
받은 데이터 길이 : 304304
받은 데이터 길이 : 305328
받은 데이터 길이 : 306352
받은 데이터 길이 : 307376
받은 데이터 길이 : 308400
받은 데이터 길이 : 309424
받은 데이터 길이 : 310448
받은 데이터 길이 : 311472
받은 데이터 길이 : 312496
받은 데이터 길이 : 313520
받은 데이터 길이 : 314544
받은 데이터 길이 : 315568
받은 데이터 길이 : 316592
받은 데이터 길이 : 317616
받은 데이터 길이 : 318640
받은 데이터 길이 : 319664
받은 데이터 길이 : 320688
받은 데이터 길이 : 321712
받은 데이터 길이 : 322736
받은 데이터 길이 : 323760
받은 데이터 길이 : 324784
받은 데이터 길이 : 325808
받은 데이터 길이 : 326832
받은 데이터 길이 : 327856
받은 데이터 길이 : 328880
받은 데이터 길이 : 329904
받은 데이터 길이 : 330928
받은 데이터 길이 : 331952
받은 데이터 길이 : 

[192.168.0.6] 접속종료
--- 대화 참여자 수 [0]
